In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dill
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, ParameterGrid, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
import xgboost

sns.set()

In [14]:
# Load previously saved preprocessed data splits
with open('../results/data_split_nsr.pkl', 'rb') as file:
    data_split_nsr = dill.load(file)

ftr_names = ['L2 prox gbar', 'L5 prox gbar', 'L2 dist gbar', 'L5 dist gbar', 'Prox variance', 'Dist variance',
               'Prox mean time', 'Dist mean time']

In [15]:
results_dict = dict()
res_names = ['linear_regression', 'ridge_regression', 'kn_regression']
regression_dict = {
    'linear_regression': LinearRegression,
    'ridge_regression': Ridge,
    'kn_regression': KNeighborsRegressor,
    'sv_regression': SVR,
    'xgb_regression': xgboost.XGBRegressor
}

for name in res_names:
    with open(f'../results/{name}_results.pkl', 'rb') as file:
        results_dict[name] = dill.load(file)

## Linear Regression Coefficients

## Permutation Test

In [18]:
def permutation_importance(X_test, y_test, reg, ftr_names):
    np.random.seed(42)
    test_score = r2_score(reg.predict(X_test), y_test)

    nr_runs = 10
    scores = np.zeros([len(ftr_names),nr_runs])
    print('test score = ',test_score)
    print('test baseline = ',np.sum(y_test == 0)/len(y_test))
    # loop through the features
    for i in range(len(ftr_names)):
        print('shuffling '+str(ftr_names[i]))
        acc_scores = []
        for j in range(nr_runs):
            X_test_shuffled = X_test.copy()
            X_test_shuffled[ftr_names[i]] = np.random.permutation(X_test[ftr_names[i]].values)
            y_pred_shuffled = reg.predict(X_test_shuffled)
            acc_scores.append(r2_score(y_pred_shuffled, y_test))
        print('   shuffled test score:',np.around(np.mean(acc_scores),3),'+/-',np.around(np.std(acc_scores),3))
        scores[i] = acc_scores

    return scores, test_score

In [1]:
model_name = 'kn_regression'
split_idx = 0
random_idx = 0

X_train = pd.DataFrame(data_split_nsr[split_idx]['X_train'], columns=ftr_names)
X_test = pd.DataFrame(data_split_nsr[split_idx]['X_test'], columns=ftr_names)
y_train, y_test = data_split_nsr[split_idx]['y_train'],  data_split_nsr[split_idx]['y_test']

reg = regression_dict[model_name]()
params = results_dict[model_name][split_idx][random_idx]['best_params']
reg.set_params(**params)
reg.fit(X_train, y_train)


NameError: name 'pd' is not defined

In [22]:
score, test_score = permutation_importance(X_test, y_test, reg, ftr_names)

test score =  0.43412827559666745
test baseline =  0.0
shuffling L2 prox gbar
   shuffled test score: -0.041 +/- 0.011
shuffling L5 prox gbar
   shuffled test score: -0.101 +/- 0.009
shuffling L2 dist gbar
   shuffled test score: 0.163 +/- 0.008
shuffling L5 dist gbar


In [ ]:
sorted_indcs = np.argsort(np.mean(scores,axis=1))[::-1]
plt.rcParams.update({'font.size': 14})
plt.figure(figsize=(8,6))
plt.boxplot(scores[sorted_indcs].T,labels=ftr_names[sorted_indcs],vert=False)
plt.axvline(test_score,label='test score')
plt.title("Permutation Importances (test set)")
plt.xlabel('score with perturbed feature')
plt.legend()
plt.tight_layout()
plt.show()

## XGBoost Feature Importances